# Python中階-讓爬蟲顯示得更美好


## 中正大學資管系 (20181021) 大綱

完成了爬蟲主程式，接著要怎樣讓顯示畫面更漂亮？下面我們使用一些格式化設定讓畫面顯示起來更好看

+ 優化顯示前
+ 優化顯示準備工作
+ 讓顯示更美好

## 優化顯示前

回到剛剛程式，在尚未優化顯示我們試用一個 `dict` 的資料結構接著印出，但的確不是很好看。因此這段我們將讓爬蟲印出的訊息更漂亮一些。
如果有興趣可以看一些這篇的[字元寬度處理](https://github.com/urwid/urwid/blob/master/urwid/old_str_util.py)

In [2]:
import requests
from requests_html import HTML


def fetch(url):
    response = requests.get(url)
    return response


def parser_article_meta(entry):
    context = {
        'title': entry.find('div.title', first=True).text,
        'push': entry.find('div.nrec', first=True).text,
        'date': entry.find('div.date', first=True).text,
        'author': entry.find('div.author', first=True).text,
    }
    return context


def main():
    resp = fetch(url='https://www.ptt.cc/bbs/movie/index.html')
    if resp.status_code == 200:
        html = HTML(html=resp.text)
        post_entries = html.find('div.r-ent')

        for entry in post_entries:
            meta = parser_article_meta(entry)
            print(meta)
    else:
        print(resp.status_code)

if __name__ == '__main__':
    main()

{'title': '[公告] 水桶公告 20181017', 'push': '2', 'date': '10/17', 'author': 'VOT1077'}
{'title': '[新聞] 登陸月球50年《電影哆啦A夢 大雄的月球探', 'push': '3', 'date': '10/17', 'author': 'hoanbeh'}
{'title': 'Re: [贈票] 【極智對決】 台北贈票', 'push': '14', 'date': '10/17', 'author': 'rapsd520'}
{'title': '新聞文章請以新發文方式-V <Reewalker>', 'push': '', 'date': '10/17', 'author': '-'}
{'title': '[請益] 李小龍傳', 'push': '4', 'date': '10/17', 'author': 'hsinofkids'}
{'title': '[討論] 最經典的系列作有哪些', 'push': '4', 'date': '10/17', 'author': 'assggy'}
{'title': '[新聞] 重建熱蘭遮城將投入135億 魏德聖：2024', 'push': '58', 'date': '10/17', 'author': 'purue'}
{'title': '[贈票] 極智對決 週六台北贈票', 'push': '', 'date': '10/17', 'author': 'WAV'}
{'title': '[情報] 2018 亞太銀幕獎 入圍名單', 'push': '', 'date': '10/17', 'author': 'qpr322'}
{'title': '[問片] 孕婦車禍流產，找人復仇的血腥片', 'push': '3', 'date': '10/17', 'author': 'shuffling'}
{'title': '[新聞] 華倫夫婦加入《安娜貝爾3》', 'push': '4', 'date': '10/17', 'author': 'shengchiu303'}
{'title': '[版規] 電影版版規 201808', 'push': '1', 'date': '8/28', 'author': 'VOT1

## 優化顯示準備工作

+ 格式化寬度，讓顯示畫面項 ptt 原本樣子
+ 建立一個格式化畫面的函式
+ 將爬取的內容帶入

In [2]:
print("學號\t姓名\t語文\t數學\t英語")
print("2017001\t曹操\t99\t88\t\t0")
print("2017002\t周瑜\t92\t45\t\t93")
print("2017008\t黃蓋\t77\t82\t\t100")

學號	姓名	語文	數學	英語
2017001	曹操	99	88		0
2017002	周瑜	92	45		93
2017008	黃蓋	77	82		100


In [3]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]

In [4]:
def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)

In [5]:
def pretty_print(push, title, date, author):
    pattern = '%3s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author))

## 讓顯示更美好

接著可能需要做一些原本程式的修改。因為原本建立一個 `main()` 函式來印出結果，但目前在印出前要做些處理，所以我們要修改一下，用 `pretty_print` 這個新的 function 來取代原本的 `print` function

In [6]:
def main():
    resp = fetch(url='https://www.ptt.cc/bbs/movie/index.html')
    if resp.status_code == 200:
        html = HTML(html=resp.text)
        post_entries = html.find('div.r-ent')

        for entry in post_entries:
            meta = parser_article_meta(entry)
            pretty_print(meta['push'], meta['title'], meta['date'], meta['author']) #取代的地方
    else:
        print(resp.status_code)

if __name__ == '__main__':
    main()

  2	[公告] 水桶公告 20181017                          10/17	VOT1077
  3	[新聞] 登陸月球50年《電影哆啦A夢 大雄的月球探     10/17	hoanbeh
 14	Re: [贈票] 【極智對決】 台北贈票                  10/17	rapsd520
   	新聞文章請以新發文方式-V <Reewalker>              10/17	-
  4	[請益] 李小龍傳                                   10/17	hsinofkids
  4	[討論] 最經典的系列作有哪些                       10/17	assggy
 58	[新聞] 重建熱蘭遮城將投入135億 魏德聖：2024       10/17	purue
  1	[贈票] 極智對決 週六台北贈票                      10/17	WAV
   	[情報] 2018 亞太銀幕獎 入圍名單                   10/17	qpr322
  3	[問片] 孕婦車禍流產，找人復仇的血腥片             10/17	shuffling
  6	[新聞] 華倫夫婦加入《安娜貝爾3》                  10/17	shengchiu303
   	[資訊]2018新北市影視人才培育課程結訓儀式          10/17	unalaba
   	[片單] 類似上海灘賭聖一片兩拍的電影?              10/17	Marchosias
   	[討論] 王家衛真喜歡用劇組人員的名字               10/17	joey0602
  1	[版規] 電影版版規 201808                          8/28	VOT1077
   	[公告] 電影版板規修訂說明                         8/28	VOT1077
 33	[公告] 關於特定影片負(好)雷討論                   10/11	VOT1077


有沒有更完整了呢？